In [2]:
import tensorflow as tf 
from sklearn import datasets  
from matplotlib import pyplot as plt 
import numpy as np 
 
#导入数据
x_data = datasets.load_iris().data #导入iris数据集的特征
 
y_data = datasets.load_iris().target #导入iris数据集的标签
 
#随机打乱顺序，使训练更具准确性
np.random.seed(120)#调用numpy中的random方法里的seed方法，赋值120，使输入特征和标签能够一一对应
 
np.random.shuffle(x_data) #调用numpy中的random方法里的shuffle方法，将训练集x_data里的特征值乱序
 
np.random.seed(120)#调用numpy中的random方法里的seed方法，赋值120，使输入特征和标签能够一一对应
 
np.random.shuffle(y_data) #调用numpy中的random方法里的shuffle方法，将测试集y_data里的标签乱序
 
tf.random.set_seed(120)#调用tensorflow中的random方法里的set_seed方法，赋值120
 
#划分数据集
x_train = x_data[:-30] #将iris数据集（特征，共150行，此时已打乱）前120行作为训练集x_train
 
y_train = y_data[:-30] #将iris数据集（标签，共150行，此时已打乱）前120行作为训练集y_train
 
x_test = x_data[-30:] #将iris数据集（特征，共150行，此时已打乱）最后30行作为测试集x_test
 
y_test = y_data[-30:] #将iris数据集（标签，共150行，此时已打乱）最后30行作为测试集y_test
 
#转换特征值的数据类型，使之与后面数据运算时数据类型一致
x_train = tf.cast(x_train, dtype = tf.float32) #调用tensorflow中的cast方法，将x_train中的特征值类型转换为float32
 
x_test = tf.cast(x_test, dtype = tf.float32) #调用tensorflow中的cast方法，将x_test中的特征值类型转换为float32
 
#用from_tensor_slices方法将特征值和标签值配对
train_data_batch = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)#将训练集的特征x_train和标签y_train配对，用batch方法将120个训练数据分成32个为一组的批次
 
test_data_batch = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)#将测试集的特征x_test和标签y_test配对，用batch方法将30个训练数据分成32个为一组的批次
 
#用truncated_normal方法构建神经网络，并用Variable方法标记可训练数据
w1 = tf.Variable(tf.random.truncated_normal([4, 3], stddev = 0.1, seed = 1))#用truncated_normal方法,构建4个输入特征，3个分类的神经网络结构，标准差为0.1的正态分布，随机种子为1
 
b1 = tf.Variable(tf.random.truncated_normal([3], stddev = 0.1, seed = 1))#用truncated_normal方法,因为b1和w1的分类维度要一样，所以是3，标准差为0.1的正态分布，随机种子为1
 
#设置变量
learnRate = 0.1 #学习率为0.1
 
train_loss_results = [] #将每轮的loss记录在此列表中，为后面画loss曲线时提供数据
 
test_accuracy = [] #将每轮的精度accuracy记录在此列表中，为后面画精度accuracy曲线提供数据
 
epoch = 500 #循环500轮
 
loss_all = 0 #每轮分4个step,loss_all记录4个step生成的4个loss的和
 
#训练部分
for epoch in range(epoch): #遍历数据集，每个epoch循环一次数据集
    for step, (x_train, y_train) in enumerate(train_data_batch): #遍历batch，每个step循环一次batch
        with tf.GradientTape() as tape: #用上下文管理器记录梯度信息
            y = tf.matmul(x_train, w1) + b1 #神经网络乘加运算，用tensorflow中的matmul方法将训练特征值x_train和w1参数进行矩阵相乘
            y = tf.nn.softmax(y) #用tensorflow中的softmax方法将神经网络乘加运算后得到的输出符合正态分布，输出和为1，可以在之后用来与独热码相减求loss
            y_one_hot = tf.one_hot(y_train, depth = 3) #用tensorflow中的one_hot方法将训练标签y_train转换为独热码格式，因为y输出为3，所以深度为3，方便接下来计算loss的和
            loss = tf.reduce_mean(tf.square(y_one_hot - y)) #用tensorflow中的reduce_mean方法求平均值，用tensorflow中的square方法求平方，这里用均方误差求损失函数loss
            loss_all += loss.numpy() #将每个step计算出的loss累加，后面可以用来求loss平均值，
 
        #计算loss对各个参数的梯度
        loss_gradient = tape.gradient(loss, [w1, b1])#用tensorflow中的GradientTape方法中的gradient方法求loss对各个参数w1,b1的梯度gradient
 
        #梯度更新
        w1.assign_sub(learnRate * loss_gradient[0]) #用assign_sub方法进行自减，实现参数w1的自动更新,等价于w1 = w1 - learn_Rate * loss_gradient[0]
        b1.assign_sub(learnRate * loss_gradient[1]) #用assign_sub方法进行自减，实现参数b1的自动更新,等价于b = b - learn_Rate * loss_gradient[1]
 
    # 每个epoch,打印loss信息
    print("epoch: {}, loss: {}".format(epoch,
                                       loss_all / 4))  # 每个epoch,打印loss信息,有4个step，所以总loss_all要除以4，求得每次step的平均loss
    train_loss_results.append(loss_all / 4)  # 用append方法将4个step的loss求平均值记录在train_loss_results中
    loss_all = 0  # loss_all归零，为下一个epoch的求loss做准备
 
    # 测试部分
    total_correct = 0  # total_correct为预测对的样本个数，初始化为0
    total_test_number = 0  # total_number为测试的总样本数，初始化为0
 
    for x_test, y_test in test_data_batch:  # 遍历训练集的特征值和标签值
        # 用更新后的参数进行预测
        y = tf.matmul(x_test, w1) + b1  # 用tensorflow中的matmul方法来进行乘加运算，再加上b1得到前向传播的结果
        y = tf.nn.softmax(y)  # 用tensorflow中的softmax方法将神经网络乘加运算后得到的前向传播的结果符合正态分布，输出和为1，可以在之后用来与独热码相减求loss
        predict = tf.argmax(y, axis=1)  # 用tensorflow中的argmax方法，返回y中最大值的索引，即预测的标签分类，axis表示按列求值
        predict = tf.cast(predict, dtype=y_test.dtype)  # 将predict的类型转换为测试集标签y_test的数据类型
        correct = tf.cast(tf.equal(predict, y_test),
                          dtype=tf.int32)  # 用tensorflow中的equal方法判断，若分类正确，则值为1，否则为0，并用tensorflow中的cast方法将bool类型转化为int32类型
        correct = tf.reduce_sum(correct)  # 用tensorflow中的reduce_sum方法将每个batch的correct数加起来
        total_correct += int(correct)  # 将所有batch中的correct数转化为int类型，并加起来
        total_test_number += x_test.shape[0]  # 用shape方法返回测试集特征x_test的行数，也就是测试的总样本数
 
    accuracy = total_correct / total_test_number  # 总的准确率
    test_accuracy.append(accuracy)  # 测试集的准确率添加到列表中来，方便记录
    print("test_accuracy:", accuracy)  # 打印测试集精度准确率
    print("-------------------------------------------------")  # 为每个epoch进行分隔，方便查看
 



epoch: 0, loss: 0.24386145547032356
test_accuracy: 0.4666666666666667
-------------------------------------------------
epoch: 1, loss: 0.2066265232861042
test_accuracy: 0.6333333333333333
-------------------------------------------------
epoch: 2, loss: 0.19424713030457497
test_accuracy: 0.6333333333333333
-------------------------------------------------
epoch: 3, loss: 0.18378842249512672
test_accuracy: 0.6333333333333333
-------------------------------------------------
epoch: 4, loss: 0.17445169016718864
test_accuracy: 0.6333333333333333
-------------------------------------------------
epoch: 5, loss: 0.16637611761689186
test_accuracy: 0.6333333333333333
-------------------------------------------------
epoch: 6, loss: 0.15946495160460472
test_accuracy: 0.6333333333333333
-------------------------------------------------
epoch: 7, loss: 0.15355950593948364
test_accuracy: 0.6333333333333333
-------------------------------------------------
epoch: 8, loss: 0.14849310368299484
test_

epoch: 72, loss: 0.07830360159277916
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 73, loss: 0.07783616054803133
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 74, loss: 0.07737472653388977
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 75, loss: 0.07691919431090355
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 76, loss: 0.07646945025771856
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 77, loss: 0.07602540124207735
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 78, loss: 0.07558693457394838
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 79, loss: 0.0751539571210742
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 80, loss: 0.07472637295722

epoch: 145, loss: 0.05524970591068268
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 146, loss: 0.05504176765680313
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 147, loss: 0.054835802875459194
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 148, loss: 0.054631791077554226
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 149, loss: 0.05442970246076584
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 150, loss: 0.054229521192610264
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 151, loss: 0.05403121467679739
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 152, loss: 0.05383475590497255
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 153, loss: 0.0

epoch: 214, loss: 0.04446378070861101
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 215, loss: 0.04434775235131383
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 216, loss: 0.04423259850591421
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 217, loss: 0.04411828750744462
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 218, loss: 0.044004823081195354
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 219, loss: 0.043892189394682646
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 220, loss: 0.043780375737696886
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 221, loss: 0.043669382110238075
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 222, loss: 0.

epoch: 287, loss: 0.03779285866767168
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 288, loss: 0.037721427623182535
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 289, loss: 0.03765042498707771
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 290, loss: 0.03757983073592186
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 291, loss: 0.03750965418294072
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 292, loss: 0.03743987949565053
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 293, loss: 0.03737052297219634
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 294, loss: 0.03730155760422349
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 295, loss: 0.037

epoch: 357, loss: 0.033639322966337204
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 358, loss: 0.03359027998521924
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 359, loss: 0.033541475888341665
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 360, loss: 0.03349291440099478
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 361, loss: 0.033444583881646395
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 362, loss: 0.03339648945257068
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 363, loss: 0.03334862529300153
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 364, loss: 0.03330099186860025
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 365, loss: 0.0

test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 429, loss: 0.030627933330833912
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 430, loss: 0.030592398950830102
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 431, loss: 0.030557010555639863
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 432, loss: 0.030521762324497104
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 433, loss: 0.030486657517030835
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 434, loss: 0.030451703583821654
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 435, loss: 0.030416883528232574
test_accuracy: 0.9666666666666667
-------------------------------------------------
epoch: 436, loss: 0.030382206663489342
test_accuracy: 0